In [28]:
print("refu bot")

refu bot


In [29]:
import os
os.chdir("C:\\Refu Uday\\refu_drive_llama\\refu_drive_llama")


In [30]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader,UnstructuredExcelLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [31]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [32]:
extracted_data = load_pdf("Data/")

In [33]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 200, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [34]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 230


In [35]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [36]:
embeddings = download_hugging_face_embeddings()

In [37]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [38]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [39]:
from dotenv import load_dotenv
load_dotenv()

True

In [40]:
PINECONE_API_KEY=os.environ.get("PINECONE_API_KEY")
GEMINI_API_KEY=os.environ.get("GEMINI_API_KEY")

In [41]:
import os
os.environ["PINECONE_API_KEY"]= PINECONE_API_KEY
os.environ["GEMINI_API_KEY"]= GEMINI_API_KEY

In [42]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)

In [43]:
index_name = "refu"

pc.create_index(
    name=index_name,
    dimension=384, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

In [44]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

In [45]:
from langchain_community.vectorstores import Pinecone

docsearch = Pinecone.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)


In [46]:
from langchain_community.vectorstores import Pinecone

# Load the existing index
docsearch = Pinecone.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)


In [47]:
docsearch

In [48]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})


In [49]:
retrieved_docs = retriever.invoke("What is Acne?")

In [50]:
retrieved_docs

[]

In [51]:
from langchain_google_genai import GoogleGenerativeAI

llm = GoogleGenerativeAI(model="gemini-pro", temperature=0.4, max_output_tokens=500)

In [52]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "give answer in paragraph max words in paragaph is 30 to 40"
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [53]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [54]:
response = rag_chain.invoke({"input": "what is Acromegaly and gigantism?"})
print(response["answer"])

Acromegaly and gigantism are both caused by excessive growth hormone production. Acromegaly occurs in adults and is characterized by overgrowth of the bones and soft tissues, particularly in the hands, feet, and face. Gigantism occurs in children and is characterized by excessive growth in height. Both conditions can lead to a variety of health problems, including diabetes, heart disease, and arthritis.
